<a href="https://colab.research.google.com/github/Javierxd1/Analisis-de-sentimientos-en-comentarios/blob/main/Aplicaci%C3%B3n%3A%20Analisis%20de%20sentimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Librerias requeridas
!pip install transformers
!pip install wordcloud

In [2]:
import re
import io

import pandas as pd
import numpy as np

from transformers import pipeline
from wordcloud import WordCloud, STOPWORDS
from PIL import Image


In [3]:
from os import sendfile
#Flujo de actividades

#Pipeline de análisis de sentimiento
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="Javierxd17/roberta-base-bne-SentimentAnalysis-cursoNLP-Platzi"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
#NER Pipeline
ner_pipeline = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    tokenizer="mrm8488/bert-spanish-cased-finetuned-ner"
)

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
#Funcionalidades

# Función para limpiar el texto
def clean(text):
    # Eliminar textos entre corchetes (ej.: etiquetas)
    text = re.sub(r'\[.*?\]', '', text)

    # Eliminar URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Eliminar etiquetas HTML
    text = re.sub(r'<.*?>+', '', text)

    # Eliminar espacios extras al inicio y final
    text = text.strip()

    return text

In [6]:
#Reconstrucción de entidades
def reconstruct_entity(ner_tokens):
    """
    Reconstruye una entidad a partir de una lista de tokens de NER.
    Si un token empieza con "##", se une al token anterior sin espacio.
    """
    entity = ""
    for token in ner_tokens:
        word = token['word']
        if word.startswith("##"):
            entity += word[2:]
        else:
            if entity:
                entity += " " + word
            else:
                entity += word
    return entity

In [7]:
# Función para procesar la salida del NER y agrupar tokens en entidades completas
def process_ner_output(ner_results):
    """
    Procesa la salida del NER ignorando el tipo de entidad y devuelve un diccionario
    con una única clave "entities" cuyo valor es la entidad reconstruida a partir de todos los tokens.
    """
    # Reconstruir la entidad a partir de todos los tokens de la lista
    combined = reconstruct_entity(ner_results)
    return {"entities": combined}

In [8]:
# Función para analizar un solo texto
def analyze_text(input_text):
    input_text = clean(input_text)
    sentiment = sentiment_pipeline(input_text)
    ner_results = ner_pipeline(input_text)
    processed_ner = process_ner_output(ner_results)
    return sentiment, processed_ner

In [9]:
# Función para analizar un archivo CSV
def analyze_csv(file_obj):
    df = pd.read_csv(file_obj.name)
    if "review_body" not in df.columns:
        return "Error: No se encontró la columna 'review_body'.", None, None
    texts = df["review_body"].astype(str).tolist()

    # Limpiar cada reseña
    cleaned_texts = [clean(text) for text in texts]

    # Obtener análisis de sentimiento y NER para cada reseña limpia
    sentiments = [sentiment_pipeline(text) for text in cleaned_texts]
    ner_all = [process_ner_output(ner_pipeline(text)) for text in cleaned_texts]

    # Extraer las entidades detectadas (valor) de cada reseña
    ner_words = []
    for ner_result in ner_all:
        # ner_result es un diccionario con la clave "entities"
        ner_words.append(ner_result["entities"])

    # Unir todas las entidades en un solo string
    combined_ner_text = " ".join(ner_words)

    # Generar wordcloud basado en las entidades detectadas
    wc = WordCloud(stopwords=STOPWORDS, background_color="white", width=800, height=400).generate(combined_ner_text)
    buf = io.BytesIO()
    wc.to_image().save(buf, format="PNG")
    buf.seek(0)

    # Convertir a imagen PIL para que Gradio lo pueda mostrar
    image = Image.open(buf)

    return sentiments, ner_all, image

##Tablero de aplicación

In [10]:
!pip install gradio

In [11]:
import gradio as gr

In [12]:
with gr.Blocks(theme=gr.themes.Citrus()) as demo:
    gr.Markdown("## Aplicación de Análisis de Reseñas de un e-commerce 📦🔍")

    with gr.Tab("Análisis de Texto"):
        gr.Markdown("### Ingrese una reseña de texto")
        text_input = gr.Textbox(label="Texto de Reseña", placeholder="Escribe aquí la reseña...")
        sentiment_output = gr.JSON(label="Análisis de Sentimiento")
        ner_output = gr.JSON(label="Entidades Reconocidas (NER)")
        analyze_btn = gr.Button("Analizar Texto")
        analyze_btn.click(analyze_text, inputs=text_input, outputs=[sentiment_output, ner_output])

    with gr.Tab("Análisis de CSV"):
        gr.Markdown("### Suba un archivo CSV con una columna 'review_body'")
        csv_input = gr.File(label="Archivo CSV")
        csv_sentiment_output = gr.JSON(label="Análisis de Sentimiento (por Reseña)")
        csv_ner_output = gr.JSON(label="Entidades Reconocidas (por Reseña)")
        wc_output = gr.Image(label="WordCloud (Entidades)")
        analyze_csv_btn = gr.Button("Analizar CSV")
        analyze_csv_btn.click(analyze_csv, inputs=csv_input, outputs=[csv_sentiment_output, csv_ner_output, wc_output])

In [13]:
#Lanzar la aplicación
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4f1e486af7053fc76d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4f1e486af7053fc76d.gradio.live
